In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
tqdm.pandas()
import os


In [ ]:
import requests
from math import nan

import pandas as pd
import csv

In [ ]:
pip install git+https://github.com/huggingface/transformers accelerate

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-wendx74y
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-wendx74y
  Resolved https://github.com/huggingface/transformers to commit 64e9b049d96e7c42abfacc44b94d1b00f7a6c7db
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.5 MB/s 

In [ ]:
!pip install transformers accelerate timm einops bitsandbytes --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 36.8 MB/s eta 0:00:00


In [ ]:
!pip install qwen-vl-utils[decord]==0.0.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 83.2 MB/s eta 0:00:00


In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)


# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

['The image depicts a serene beach scene with a person and a dog. The person is sitting on the sandy beach, facing the ocean. They are wearing a plaid shirt and black pants, and they have long hair. The dog, which appears to be a Labrador Retriever, is sitting on the sand and is interacting with the person by placing its paw on their hand. The dog is wearing a harness with a colorful pattern. The background shows the ocean with gentle waves and a clear sky, suggesting it might be early morning or late afternoon due to the soft lighting. The overall atmosphere of the image is peaceful and joyful.']


In [ ]:
# from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
# import torch
# from PIL import Image
# import requests

# processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

# model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True)
# model.to("cuda:0")




# **THIS IS FOR ML DATASET**

In [ ]:

# Load link.csv
links = pd.read_csv('links.csv')

# Your TMDb API key
api_key = 'f404e677eeb220edf77cbe227f4ae0db'

# Function to fetch poster URL
def get_poster(tmdb_id):
    url = f'https://api.themoviedb.org/3/movie/{tmdb_id}?api_key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        poster_path = data.get('poster_path')
        if poster_path:
            return f'https://image.tmdb.org/t/p/w342{poster_path}'
    return None




In [ ]:
links['poster_url'] = links['tmdbId'].progress_apply(get_poster)

  0%|          | 0/9742 [00:00<?, ?it/s]

In [ ]:
os.makedirs("product_images", exist_ok=True)

for _, row in links.iterrows():
    url = row['poster_url']
    if pd.notna(url):
        img_data = requests.get(url).content
        with open(f"product_images/{row['movieId']}.jpg", 'wb') as f:
            f.write(img_data)

# **THIS IS FOR QWEN**

In [ ]:
def getDescribe(link):
    # messages = [
    #     {
    #         "role": "user",
    #         "content": [
    #             {
    #                 "type": "image",
    #                 "image": link,
    #             },
    #             {"type": "text", "text": "Assuming you are a recommender system, provided the images of a product in an online platform. Describe this product in 2-3 sentence that is input of retrieval model."},
    #         ],
    #     }
    # ]
    messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": link,
            },
            {
                "type": "text",
                "text": (
                    "You are a recommender system. Given the product image, generate a concise and keyword-rich description in 2–3 sentences. "
"This description will be used as input to a retrieval model (e.g., BM25) and should highlight visual features that meaningfully represent the product. "
"Include specific object types, colors, materials, patterns that are visually observable and relevant for recommending similar items."

                )
            },
        ],
    }
    ]
    # messages = [
    # {
    #     "role": "user",
    #     "content": [
    #         {
    #             "type": "image",
    #             "image": link,
    #         },
    #         {
    #             "type": "text",
    #             "text": (
    #                 "You are a recommender system. Given the product image, Given the product image, extract 5 relevant and concise keywords that best describe the product "
    #                 "for retrieval purposes (e.g., BM25). Focus on key or outstanding visual attributes such as object type, color, material, style, or usage context."

    #             )
    #         },
    #     ],
    # }
    # ]
#     messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "image": link,
#             },
#             {
#                 "type": "text",
#                 "text": (
#                     "You are a recommender system. Given this product image, think about how you would describe it in general terms for an online platform. "
#                     "Then, based on that general description, output exactly 5 keywords that best capture its essential features for a retrieval system (e.g., BM25). "
#                     "Only output the keywords, separated by commas."
#                 )
#             },
#         ],
#     }
# ]


    # Preparation for inference
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return output_text

In [ ]:
getDescribe("https://images-na.ssl-images-amazon.com/images/I/51ptWXqrj3L.jpg")

['The image showcases a white ResponseCard RF LCD device from Turning Technologies, featuring a grid of black buttons with clear labels such as 1/A, 2/B, 3/C, 4/D, 5/E, 6/F, 7/G, 8/H, 9/I, Ch, and 0/J. The device has a small screen at the top displaying "Turning Technologies" and "ResponseCard RF LCD." It is placed on a patterned carpet with a mix of brown, beige, and red hues. The website www.TurningTechnologies.com is visible at the bottom.']

In [ ]:
getDescribe("https://images-na.ssl-images-amazon.com/images/I/51ptWXqrj3L._SX38_SY50_CR,0,0,38,50_.jpg")

['RFID, Access Control, Security, Reader, Identification']

# **AMAZON**

In [ ]:
import gzip
import json


In [ ]:
data = []
with gzip.open("/content/meta_Industrial_and_Scientific.json.gz", "r") as f:
  for line in f:
    data.append(json.loads(line))

df = pd.DataFrame(data)

In [ ]:
df.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')

In [ ]:
df_filtered = df[["asin", "title", "feature", "description", "imageURL", "imageURLHighRes", "category"]].copy()

In [ ]:
unique_Meta_asin = df_filtered['asin'].unique()
print(f"Number of unique ASINs: {len(unique_Meta_asin)}")


Number of unique ASINs: 165687


In [ ]:
empty_imageurl_count = df_filtered[df_filtered['imageURLHighRes'].apply(lambda x: not x)].shape[0]
print(f"Number of unique ASINs with empty imageURL: {empty_imageurl_count}")


Number of unique ASINs with empty imageURL: 99631


In [ ]:
data = []
with gzip.open("/content/Industrial_and_Scientific_5.json.gz", "r") as f:
  for line in f:
    data.append(json.loads(line))

df = pd.DataFrame(data)
print(df.columns)

unique_asin = df['asin'].unique()
print(f"Number of unique ASINs: {len(unique_asin)}")


Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image'],
      dtype='object')
Number of unique ASINs: 5334


In [ ]:
overlapping_asin = set(unique_Meta_asin).intersection(set(unique_asin))
len(overlapping_asin)

5327

In [ ]:
# rm -rf product_images/

In [ ]:
# prompt: crawl all images from imageURL column
counter = 0
counter_nolist = 0
counter_listEmpty = 0
os.makedirs(f"product_images", exist_ok=True)
for cnt, row in tqdm(df_filtered.iterrows(), total =  len(df_filtered)):
  image_urls = row['imageURLHighRes']
  asin = row['asin']
  if asin not in overlapping_asin:
    continue
  if os.path.exists(f"product_images/{asin}.jpg"):
    continue

  if isinstance(image_urls, list):
    if len(image_urls) == 0:
      counter_listEmpty += 1
      continue
    for i, url in enumerate(image_urls):
      try:
        img_data = requests.get(url).content
        with open(f"product_images/{asin}.jpg", 'wb') as f:
          f.write(img_data)
        break
      except Exception as e:
        counter += 1
  else:
    counter_nolist += 1

print(f"missing: {counter}, not in list form: {counter_nolist}, list but empty: {counter_listEmpty}")

num_files = len(os.listdir("product_images"))
print(f"Number of files in product_images: {num_files}")

  0%|          | 0/167442 [00:00<?, ?it/s]

missing: 0, not in list form: 0, list but empty: 1940
Number of files in product_images: 3491


In [ ]:
!zip -r product_images.zip product_images

In [ ]:
image_directory = "product_images"
image_files = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]
len(image_files)

3491

In [ ]:
# prompt: from the downloaded product images in product_images directory, get the path corresponding the asin in unique asin, put the path of each images in that directory into the getDescribe

# Create a dictionary to map ASINs to image paths
asin_image_paths = {}
tmp = []
for image_path in image_files:
    # Assuming the filename is the ASIN followed by .jpg
    asin = os.path.splitext(os.path.basename(image_path))[0]
    tmp.append(asin)
    asin_image_paths[asin] = image_path

# Create a dictionary to store the descriptions for unique ASINs
asin_descriptions = {}

In [ ]:
tmp = set(tmp)

In [ ]:
len(asin_image_paths)

3491

In [ ]:
counter = 0

# Iterate through the unique ASINs and get descriptions for corresponding images
for asin in tqdm(unique_asin):
    if asin in asin_descriptions:
        continue
    if asin in asin_image_paths:
        image_path = asin_image_paths[asin]
        try:
            description = getDescribe(image_path)
            # getDescribe returns a list, take the first element
            asin_descriptions[asin] = description[0] if description else nan
            counter += 1
        except Exception as e:
            print(f"Error processing ASIN {asin}: {e}")
            asin_descriptions[asin] = nan
    else:
        asin_descriptions[asin] = nan # ASIN not found in downloaded images

# Convert the results to a list of tuples for writing to CSV
amazon_res = [(asin, desc) for asin, desc in asin_descriptions.items()]

# Write the results to a CSV file
amazon_output_filename = 'amazon_descriptions.csv'
with open(amazon_output_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['asin', 'description']) # Write header
    writer.writerows(amazon_res)

print(f"Amazon descriptions saved to {amazon_output_filename}")

  0%|          | 0/5334 [00:00<?, ?it/s]

In [ ]:
# prompt: from the downloaded product images in product_images directory, put the path of each images in that directory into the getDescribe

descriptions_dict = {}
counter = 0
for asin in tqdm(unique_asin):
    image_path
    asin = os.path.splitext(os.path.basename(image_path))[0] # Extract asin from filename
    try:
        description = getDescribe(image_path)
        descriptions_dict[asin] = description
    except Exception as e:
        descriptions_dict[asin] = nan

output_filename = 'amazon_product_descriptions.csv'
with open(output_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['asin', 'description']) # Write header
    for asin, description in descriptions_dict.items():
        writer.writerow([asin, description])

print(f"Amazon product image descriptions saved to {output_filename}")

In [ ]:
# prompt: from the saved poster, print link and put in the getDescribe

import pandas as pd
for idx, row in tqdm(links.iterrows()):
    if row["movieId"] in fullR:
        continue
    poslink = f"posters/{row['movieId']}.jpg"
    try:
        description = getDescribe(poslink)
        fullR[row['movieId']] = description
    except:
        fullR[row['movieId']] = nan